In [1]:
import pandas as pd
import numpy as np
import os
import datetime

# Options, overview and basic Functions

In [2]:
pd.options.display.max_columns = 500

In [3]:
df = pd.read_csv('file_with_random_transaction.csv')

In [4]:
df.head()

,id,transaction_date,direction,customer_id,tr_amount,type
0,0,2018-10-15 09:59:18.000,-1,1,28.77,transfer
1,1,2018-10-25 06:54:50.000,-1,1,35.32,transfer
2,2,2018-11-23 09:00:15.115,-1,1,56.33,pin and contacless
3,3,2018-08-13 00:59:18.000,-1,1,80.78,transfer
4,4,2018-10-19 07:02:53.000,-1,1,31.38,transfer


In [5]:
df.isnull().sum()

id                  0
transaction_date    0
direction           0
customer_id         0
tr_amount           0
type                0
dtype: int64

### Basic Functions

Some of these functions are used some not, but for me is a must to have when I work with feature engineering in python

In [6]:
def count(x):
    return x.count()

def count_unique(x):
    return x.nunique()

def max_before_last(x):
      return np.max(x.iloc[:-1])

def last_value(s):
      return s.iat[-1]
    
def first_value(s):
      return s.iat[0]
    
def count_pos(x):
    return np.sum(np.where(x > 0, 1, 0))

def count_neg(x):
    return np.sum(np.where(x < 0, 1, 0))

def sum_pos(x):
    return np.sum(np.where(x > 0, x, 0))

def sum_neg(x):
    return np.sum(np.where(x < 0, -x, 0))

# Class Aggregation - Feat Engineering

Using a fake dataset of transactions that I create I want to give an example of

- Aggregation
- Feature Engineering (this could be endless)
- Preparation for an RFM or any other analytic tool aggregate on time and/or user (month or week in this case)
- Pandas data manipulation
- Preparation for a hypothetical churn model

The class will have in input
    - The file
    - format of file (CSV and parquet accepted, but easy implement others)
    - Aggregation type (month or week accepted, but be brave and creative)
    - Select a period start and end (or none)
    
And the class will do in order
    - Read the file
    - Create some feature engineering (be inspired from it but follow the business needs and your creativity)
    - Aggregate and do extra feature engineering
    - Create a calendar (weekly or monthly) based on first and last timestamp in the data frame, and will fill each time period with customer's statistic
    - Create a flat historical data frame
    - Save in a specific folder the data frame(s)

In [7]:
class Aggregation_Transaction:

    def __init__(self, file_name, type_format, agg='month', limit_date_start = None, limit_date_end = None):
        self.file_name = file_name
        self.type_format = type_format
        self.agg = agg
        self.limit_date_start = limit_date_start
        self.limit_date_end = limit_date_end

        self.read_data(file_name, type_format)

        self.preprocessing()
        self.aggregate_data(self.agg)

    def read_data(self, file_name, type_format):
        if type_format == 'csv':
            self.df_raw = pd.read_csv(file_name)
        elif type_format == 'parquet':
            self.df_raw = pd.read_parquet(file_name)
        else:
            print('dont know this file format')


#------------------------------------------------------------
# Initial Feat Eng *** Initial Feat Eng *** Initial Feat Eng
#------------------------------------------------------------
    def preprocessing(self):#Feat eng. part I

        self.df = self.df_raw.copy(deep=True)
        #check if we have duplicates
        self.df = self.df.drop_duplicates(subset = ['id', 'customer_id'])
        self.df = (
            self.df
            .sort_values(['transaction_date', 'customer_id'])
            .assign(
                    date = lambda x : x.transaction_date.astype("datetime64[D]"), #Trunc date at day level
                    week = lambda x : x.transaction_date.astype("datetime64[W]"), #Trunc date at week level
                    month = lambda x : x.transaction_date.astype("datetime64[M]"), #Trunc date at month level
                    day_of_the_week = lambda x : x.transaction_date.astype("datetime64[D]").apply(lambda y : y.strftime('%A')), #Get the day of the week (Monday, Tuesday ...)
                    days_since_first_tr = lambda x: ((x.date - x.groupby(['customer_id']).date.transform('min')) / np.timedelta64(1, 'D')).astype(np.int),
                    months_since_first_tr = lambda x: ((x.date - x.groupby(['customer_id']).date.transform('min')) / np.timedelta64(1, 'M')).astype(np.int),
                    weeks_since_first_tr = lambda x: ((x.date - x.groupby(['customer_id']).date.transform('min')) / np.timedelta64(1, 'W')).astype(np.int),
                    days_to_next = lambda x: (x.sort_values('transaction_date', ascending=False).groupby(['customer_id']).date.diff()),
                    days_from_previous = lambda x: (x.sort_values('transaction_date', ascending=True).groupby(['customer_id']).date.diff()),
                    is_last_transaction = lambda x: np.where((x.days_to_next.isnull()) | (x.days_to_next == 0), 1, 0),
                    is_first_transaction = lambda x: np.where((x.days_from_previous.isnull()) | (x.days_from_previous == 0), 1, 0),
                    tr_amount_direction = lambda x: x.direction * x.tr_amount,
                    balance = lambda x: x.sort_values(['transaction_date']).groupby(['customer_id']).tr_amount_direction.cumsum(),
                    is_more_than_100_in = lambda x : np.where(x.tr_amount_direction < -100, 1, 0),
                    is_more_than_500_in = lambda x : np.where(x.tr_amount_direction < -500, 1, 0),
                    is_more_than_1000_in = lambda x : np.where(x.tr_amount_direction < -1000, 1, 0)
            ).assign(
                    months_since_first_tr = lambda x: np.where(x.months_since_first_tr < 1, 1, x.months_since_first_tr),
                    days_since_first_tr = lambda x: np.where(x.days_since_first_tr < 1, 1, x.days_since_first_tr)
                     ).drop(columns = ['tr_amount']).rename(columns = {'tr_amount_direction':'amount'})

            )

        if self.limit_date_start == None and self.limit_date_end == None:
            self.df = self.df
        elif self.limit_date_start != None and self.limit_date_end == None:
            self.df = self.df[self.df.month >= self.limit_date_start]
        elif self.limit_date_start == None and self.limit_date_end != None:
            self.df = self.df[self.df.month < self.limit_date_end]
        elif self.limit_date_start != None and self.limit_date_end != None:
            self.df = self.df[(self.df.month >= self.limit_date_start) & (self.df.month < self.limit_date_end)]



#-------------------------------------------------------------
# ******** Aggregation and Second Feature Engineering ********
#-------------------------------------------------------------

    def aggregate_data(self, agg):#Aggregation

#--------------------------------------------------------------
# Month Aggregation *** Month Aggregation *** Month Aggregation
#--------------------------------------------------------------

        if agg == 'month':
            self.df = (
                self.df
                .sort_values(['month', 'transaction_date'])
                .assign(
                    date_first = lambda x : x.groupby(['customer_id']).transaction_date.head(1),
                    date_2 = lambda x : x.date,
                    #General Transaction
                    amount_in = lambda x: np.where(x.amount > 0, x.amount, 0),
                    amount_out = lambda x: np.where(x.amount < 0, -x.amount, 0),
                    count_in = lambda x: np.where(x.amount > 0, 1, 0),
                    count_out = lambda x: np.where(x.amount < 0, 1, 0),
                    cum_sum_in = lambda x: x.groupby(['customer_id']).amount_in.cumsum(),
                    cum_sum_out = lambda x: x.groupby(['customer_id']).amount_out.cumsum(),
                    cum_count_in = lambda x: x.groupby(['customer_id']).count_in.cumsum(),
                    cum_count_out = lambda x: x.groupby(['customer_id']).count_out.cumsum(),
                    # Level of usage
                    cum_is_more_than_100_in = lambda x : x.groupby(['customer_id']).is_more_than_100_in.cumsum(),
                    cum_is_more_than_500_in = lambda x : x.groupby(['customer_id']).is_more_than_500_in.cumsum(),
                    cum_is_more_than_1000_in = lambda x : x.groupby(['customer_id']).is_more_than_1000_in.cumsum(),
                    #Days Bins
                    monday = lambda x : np.where(x.day_of_the_week=='Monday', 1, 0),
                    tuesday = lambda x : np.where(x.day_of_the_week=='Tuesday', 1, 0),
                    wednesday = lambda x : np.where(x.day_of_the_week=='Wednesday', 1, 0),
                    thursday = lambda x : np.where(x.day_of_the_week=='Thursday', 1, 0),
                    friday = lambda x : np.where(x.day_of_the_week=='Friday', 1, 0),
                    saturday = lambda x : np.where(x.day_of_the_week=='Saturday', 1, 0),
                    sunday = lambda x : np.where(x.day_of_the_week=='Sunday', 1, 0),
                    #Cumulative Hours
                    cum_monday = lambda x : x.groupby(['customer_id']).monday.cumsum(),
                    cum_tuesday = lambda x : x.groupby(['customer_id']).tuesday.cumsum(),
                    cum_wednesday = lambda x : x.groupby(['customer_id']).wednesday.cumsum(),
                    cum_thursday = lambda x : x.groupby(['customer_id']).thursday.cumsum(),
                    cum_friday = lambda x : x.groupby(['customer_id']).friday.cumsum(),
                    cum_saturday = lambda x : x.groupby(['customer_id']).saturday.cumsum(),
                    cum_sunday = lambda x : x.groupby(['customer_id']).sunday.cumsum()
                    )
                .groupby(['customer_id', 'month'], as_index=False) 
                .agg({
                    'date': [last_value],
                    'date_2':[first_value],
                    'date_first': [first_value],
                    'days_to_next': [np.min],
                    'amount_in': [np.sum],
                    'amount_out': [np.sum],
                    'cum_sum_in': [last_value],
                    'cum_sum_out': [last_value],
                    'count_in': [np.sum],
                    'count_out': [np.sum],
                    'cum_count_in': [last_value],
                    'cum_count_out': [last_value],
                    'balance': [last_value],
                    'type':count_unique,
                    'is_first_transaction': [np.max],
                    'is_last_transaction': [np.max],
                    'is_more_than_100_in' : [np.sum],
                    'is_more_than_500_in': [np.sum],
                    'is_more_than_1000_in': [np.sum],
                    'cum_is_more_than_100_in' : [last_value],
                    'cum_is_more_than_500_in': [last_value],
                    'cum_is_more_than_1000_in': [last_value],
                    'cum_monday': [last_value],
                    'monday': [np.sum],
                    'cum_tuesday': [last_value],
                    'tuesday': [np.sum],
                    'cum_wednesday': [last_value],
                    'wednesday':[np.sum],
                    'cum_thursday': [last_value],
                    'thursday':[np.sum],
                    'cum_friday': [last_value],
                    'friday':[np.sum],
                    'cum_saturday': [last_value],
                    'saturday':[np.sum],
                    'cum_sunday': [last_value],
                    'sunday':[np.sum],
                    })
                )
            self.df.columns = ['customer_id', 'month', 'date_last', 'first_date_month',
                          'date_first',
                          'days_to_next_max',
                          'sum_in', 'sum_out', 'cum_sum_in', 'cum_sum_out',
                          'count_in', 'count_out', 'cum_count_in', 'cum_count_out',
                          'balance', 'count_tnx_type',
                          'month_of_first_transaction', 'month_of_last_transaction',
                          'count_incoming_bigger_than_100', 'count_incoming_bigger_than_500', 'count_incoming_bigger_than_1000',
                          'cum_n_incoming_bigger_than_100', 'cum_n_incoming_bigger_than_500', 'cum_n_incoming_bigger_than_1000',
                          'cum_monday', 'monday', 'cum_tuesday', 'tuesday', 'cum_wednesday', 'wednesday', 'cum_thursday',
                          'thursday', 'cum_friday', 'friday', 'cum_saturday', 'saturday', 'cum_sunday', 'sunday']

            self.df = (self.df.sort_values(['month', 'customer_id' ])
                       .assign(
                            days_to_next_max = lambda x : x.days_to_next_max.dt.days.abs(),
                            sum_in_relative = lambda x: x.sum_in / (x.sum_in + x.sum_out),
                            sum_out_relative = lambda x: x.sum_out / (x.sum_in + x.sum_out),
                            count_in_relative = lambda x: x.count_in / (x.count_in + x.count_out),
                            count_out_relative = lambda x: x.count_out / (x.count_in + x.count_out),
                            churn = lambda x: np.where(x.count_in + x.count_out == 0, 1, 0)))

            dat = self.df.copy(deep=True)

            date_min, date_max = self.df.month.agg([np.min, np.max])
            df_dates = pd.DataFrame(pd.date_range(start=date_min, end=date_max, freq='MS'), columns=['month']).assign(ij=1)
            df_refs = pd.DataFrame({'customer_id': self.df.customer_id.unique()}).assign(ij=1)
            df_refs.merge(df_dates, on = 'ij', how = 'outer').merge
            df_rm = dat.groupby('customer_id', as_index=False).agg({'month': np.min}).rename(columns={'month':'month_min'})
            self.dfs = df_refs.merge(df_dates, on='ij', how='outer').merge(df_rm, how='left', left_on=['customer_id'], right_on=['customer_id'])

            self.dfs = (
            self.dfs[self.dfs.month >= self.dfs.month_min]
            .merge(self.df, how='left', on=['customer_id', 'month'])
            .sort_values(['month'])
            .assign(
                n_months = lambda x: x.groupby('customer_id').ij.cumsum(),
                date_last = lambda x: x.groupby('customer_id').date_last.fillna(method='ffill'),
                date_first = lambda x: x.groupby('customer_id').date_first.fillna(method='ffill'),
                days_in_month = lambda x : x.month.dt.daysinmonth,
                days_to_next_max = lambda x : x.groupby('customer_id').days_to_next_max.fillna(method='ffill'),
                days_since_last_at_end_of_the_month = lambda x: ((x.month + np.timedelta64(1, 'M') - x.date_last) / np.timedelta64(1, 'D')).astype(np.int),
                days_since_first = lambda x : ((x.month + np.timedelta64(1, 'M') - x.date_last.iloc[0])/ np.timedelta64(1, 'D')).astype(np.int),
                sum_in = lambda x : x.groupby('customer_id').sum_in.fillna(0),
                sum_out = lambda x : x.groupby('customer_id').sum_out.fillna(0),
                cum_sum_in = lambda x: x.groupby('customer_id').cum_sum_in.fillna(method='ffill'),
                cum_sum_out = lambda x: x.groupby('customer_id').cum_sum_out.fillna(method='ffill'),
                count_in = lambda x : x.groupby('customer_id').count_in.fillna(0),
                count_out = lambda x : x.groupby('customer_id').count_out.fillna(0),
                cum_count_in = lambda x: x.groupby('customer_id').cum_count_in.fillna(method='ffill'),
                cum_count_out = lambda x: x.groupby('customer_id').cum_count_out.fillna(method='ffill'),
                cum_count = lambda x : (x.cum_count_in + x.cum_count_out),
                balance = lambda x : x.cum_sum_in - x.cum_sum_out,
                count_tnx_type = lambda x : x.groupby('customer_id').count_tnx_type.fillna(0),
                churn = lambda x: np.where(x.count_in + x.count_out == 0, 1, 0),
                sum_in_relative = lambda x : x.groupby('customer_id').sum_in_relative.fillna(0),
                sum_out_relative = lambda x : x.groupby('customer_id').sum_out_relative.fillna(0),
                count_in_relative = lambda x : x.groupby('customer_id').count_in_relative.fillna(0),
                count_out_relative = lambda x : x.groupby('customer_id').count_out_relative.fillna(0),
                month_of_first_transaction = lambda x : x.groupby('customer_id').month_of_first_transaction.fillna(0),
                month_of_last_transaction = lambda x : x.groupby('customer_id').month_of_last_transaction.fillna(0),
                n_transaction_monthly = lambda x : (x.count_in + x.count_out),
                active_months = lambda x : np.where(x.n_transaction_monthly > 0, 1, 0),
                count_incoming_bigger_than_100 = lambda x : x.groupby('customer_id').count_incoming_bigger_than_100.fillna(0),
                count_incoming_bigger_than_500 = lambda x : x.groupby('customer_id').count_incoming_bigger_than_500.fillna(0),
                count_incoming_bigger_than_1000 = lambda x : x.groupby('customer_id').count_incoming_bigger_than_1000.fillna(0),
                cum_n_incoming_bigger_than_100 = lambda x : x.groupby('customer_id').cum_n_incoming_bigger_than_100.fillna(method='ffill'),
                cum_n_incoming_bigger_than_500 = lambda x : x.groupby('customer_id').cum_n_incoming_bigger_than_500.fillna(method='ffill'),
                cum_n_incoming_bigger_than_1000 = lambda x : x.groupby('customer_id').cum_n_incoming_bigger_than_1000.fillna(method='ffill'),
                cum_monday = lambda x: x.groupby('customer_id').cum_monday.fillna(method='ffill'),
                monday = lambda x : x.groupby('customer_id').monday.fillna(0),
                cum_tuesday = lambda x: x.groupby('customer_id').cum_tuesday.fillna(method='ffill'),
                tuesday = lambda x : x.groupby('customer_id').tuesday.fillna(0),
                cum_wednesday = lambda x: x.groupby('customer_id').cum_wednesday.fillna(method='ffill'),
                wednesday = lambda x : x.groupby('customer_id').wednesday.fillna(0),
                cum_thursday = lambda x: x.groupby('customer_id').cum_thursday.fillna(method='ffill'),
                thursday = lambda x : x.groupby('customer_id').thursday.fillna(0),
                cum_friday = lambda x: x.groupby('customer_id').cum_friday.fillna(method='ffill'),
                friday = lambda x : x.groupby('customer_id').friday.fillna(0),
                cum_saturday = lambda x: x.groupby('customer_id').cum_saturday.fillna(method='ffill'),
                saturday = lambda x : x.groupby('customer_id').saturday.fillna(0),
                cum_sunday = lambda x: x.groupby('customer_id').cum_sunday.fillna(method='ffill'),
                sunday = lambda x : x.groupby('customer_id').sunday.fillna(0)
                )
            .drop(columns=['ij', 'month_min']))

            #organise df
            new_order = ['customer_id', 'month','n_months',
                 'days_to_next_max', 'days_since_last_at_end_of_the_month', 'days_since_first', #days
                 'sum_in', 'sum_out','cum_sum_in', 'cum_sum_out', 'count_in', 'count_out', 'cum_count_in','cum_count_out','cum_count','n_transaction_monthly','balance', #transaction and money
                 'count_incoming_bigger_than_100', 'count_incoming_bigger_than_500', 'count_incoming_bigger_than_1000', #big transaction monthly
                 'cum_n_incoming_bigger_than_100', 'cum_n_incoming_bigger_than_500', 'cum_n_incoming_bigger_than_1000', #big transaction cumulative
                 'cum_monday', 'monday', 'cum_tuesday', 'tuesday', 'cum_wednesday', 'wednesday', 'cum_thursday',#days of the week
                 'thursday', 'cum_friday', 'friday', 'cum_saturday', 'saturday', 'cum_sunday', 'sunday',
                 'sum_in_relative', 'sum_out_relative', 'count_in_relative','count_out_relative', #relative transaction
                 'count_tnx_type', #transaction type
                 'month_of_first_transaction', 'month_of_last_transaction', #month
                 'churn','active_months', #churn/active
                 'days_in_month']

            self.df_month = self.dfs[new_order].round(2)

#-----------------------------------------------------------
# Week Aggregation *** Week Aggregation *** Week Aggregation
#-----------------------------------------------------------

        if agg == 'week':
            self.df = (
                self.df
                .sort_values(['week', 'transaction_date'])
                .assign(
                    date_first = lambda x : x.groupby(['customer_id']).transaction_date.head(1),
                    date_2 = lambda x : x.date,
                    #General Transaction
                    amount_in = lambda x: np.where(x.amount > 0, x.amount, 0),
                    amount_out = lambda x: np.where(x.amount < 0, -x.amount, 0),
                    count_in = lambda x: np.where(x.amount > 0, 1, 0),
                    count_out = lambda x: np.where(x.amount < 0, 1, 0),
                    cum_sum_in = lambda x: x.groupby(['customer_id']).amount_in.cumsum(),
                    cum_sum_out = lambda x: x.groupby(['customer_id']).amount_out.cumsum(),
                    cum_count_in = lambda x: x.groupby(['customer_id']).count_in.cumsum(),
                    cum_count_out = lambda x: x.groupby(['customer_id']).count_out.cumsum(),
                    # Level of usage
                    cum_is_more_than_100_in = lambda x : x.groupby(['customer_id']).is_more_than_100_in.cumsum(),
                    cum_is_more_than_500_in = lambda x : x.groupby(['customer_id']).is_more_than_500_in.cumsum(),
                    cum_is_more_than_1000_in = lambda x : x.groupby(['customer_id']).is_more_than_1000_in.cumsum(),
                    #Days Bins
                    monday = lambda x : np.where(x.day_of_the_week=='Monday', 1, 0),
                    tuesday = lambda x : np.where(x.day_of_the_week=='Tuesday', 1, 0),
                    wednesday = lambda x : np.where(x.day_of_the_week=='Wednesday', 1, 0),
                    thursday = lambda x : np.where(x.day_of_the_week=='Thursday', 1, 0),
                    friday = lambda x : np.where(x.day_of_the_week=='Friday', 1, 0),
                    saturday = lambda x : np.where(x.day_of_the_week=='Saturday', 1, 0),
                    sunday = lambda x : np.where(x.day_of_the_week=='Sunday', 1, 0),
                    #Cumulative Hours
                    cum_monday = lambda x : x.groupby(['customer_id']).monday.cumsum(),
                    cum_tuesday = lambda x : x.groupby(['customer_id']).tuesday.cumsum(),
                    cum_wednesday = lambda x : x.groupby(['customer_id']).wednesday.cumsum(),
                    cum_thursday = lambda x : x.groupby(['customer_id']).thursday.cumsum(),
                    cum_friday = lambda x : x.groupby(['customer_id']).friday.cumsum(),
                    cum_saturday = lambda x : x.groupby(['customer_id']).saturday.cumsum(),
                    cum_sunday = lambda x : x.groupby(['customer_id']).sunday.cumsum(),
                    )
                .groupby(['customer_id', 'week'], as_index=False)
                .agg({
                    'date': [last_value],
                    'date_2':[first_value],
                    'date_first': [first_value],
                    'days_to_next': [np.min],
                    'amount_in': [np.sum],
                    'amount_out': [np.sum],
                    'cum_sum_in': [last_value],
                    'cum_sum_out': [last_value],
                    'count_in': [np.sum],
                    'count_out': [np.sum],
                    'cum_count_in': [last_value],
                    'cum_count_out': [last_value],
                    'balance': [last_value],
                    'type':count_unique,
                    'is_first_transaction': [np.max],
                    'is_last_transaction': [np.max],
                    'is_more_than_100_in' : [np.sum],
                    'is_more_than_500_in': [np.sum],
                    'is_more_than_1000_in': [np.sum],
                    'cum_is_more_than_100_in' : [last_value],
                    'cum_is_more_than_500_in': [last_value],
                    'cum_is_more_than_1000_in': [last_value],
                    'cum_monday': [last_value],
                    'monday': [np.sum],
                    'cum_tuesday': [last_value],
                    'tuesday': [np.sum],
                    'cum_wednesday': [last_value],
                    'wednesday':[np.sum],
                    'cum_thursday': [last_value],
                    'thursday':[np.sum],
                    'cum_friday': [last_value],
                    'friday':[np.sum],
                    'cum_saturday': [last_value],
                    'saturday':[np.sum],
                    'cum_sunday': [last_value],
                    'sunday':[np.sum],
                    })
                )
            self.df.columns = ['customer_id', 'week', 'date_last', 'first_date_week',
                          'date_first',
                          'days_to_next_max',
                          'sum_in', 'sum_out', 'cum_sum_in', 'cum_sum_out',
                          'count_in', 'count_out', 'cum_count_in', 'cum_count_out',
                          'balance', 'count_tnx_type',
                          'week_of_first_transaction', 'week_of_last_transaction',
                          'count_incoming_bigger_than_100', 'count_incoming_bigger_than_500', 'count_incoming_bigger_than_1000',
                          'cum_n_incoming_bigger_than_100', 'cum_n_incoming_bigger_than_500', 'cum_n_incoming_bigger_than_1000',
                          'cum_monday', 'monday', 'cum_tuesday', 'tuesday', 'cum_wednesday', 'wednesday', 'cum_thursday',
                          'thursday', 'cum_friday', 'friday', 'cum_saturday', 'saturday', 'cum_sunday', 'sunday']


            self.df = (self.df.sort_values(['week', 'customer_id' ])
                       .assign(
                            days_to_next_max = lambda x : x.days_to_next_max.dt.days.abs(),
                            sum_in_relative = lambda x: x.sum_in / (x.sum_in + x.sum_out),
                            sum_out_relative = lambda x: x.sum_out / (x.sum_in + x.sum_out),
                            count_in_relative = lambda x: x.count_in / (x.count_in + x.count_out),
                            count_out_relative = lambda x: x.count_out / (x.count_in + x.count_out),
                            churn = lambda x: np.where(x.count_in + x.count_out == 0, 1, 0))
)

            dat = self.df.copy(deep=True)

            date_min, date_max = self.df.week.agg([np.min, np.max])
            df_dates = pd.DataFrame(pd.date_range(start=date_min, end=date_max, freq="W-THU"), columns=['week']).assign(ij=1)
            df_refs = pd.DataFrame({'customer_id': self.df.customer_id.unique()}).assign(ij=1)
            df_refs.merge(df_dates, on = 'ij', how = 'outer').merge
            df_rm = dat.groupby('customer_id', as_index=False).agg({'week': np.min}).rename(columns={'week':'week_min'})
            self.dfs = df_refs.merge(df_dates, on='ij', how='outer').merge(df_rm, how='left', left_on=['customer_id'], right_on=['customer_id'])

            self.dfs = (
            self.dfs[self.dfs.week >= self.dfs.week_min]
            .merge(self.df, how='left', on=['customer_id', 'week'])
            .sort_values(['week'])
            .assign(
                n_week = lambda x: x.groupby('customer_id').ij.cumsum(),
                month = lambda x : x.week.astype("datetime64[M]"),
                date_last = lambda x: x.groupby('customer_id').date_last.fillna(method='ffill'),
                date_first = lambda x: x.groupby('customer_id').date_first.fillna(method='ffill'),
                days_in_month = lambda x : x.month.dt.daysinmonth,
                days_to_next_max = lambda x : x.groupby('customer_id').days_to_next_max.fillna(method='ffill'),
                days_since_last_at_end_of_the_week = lambda x: ((x.week + np.timedelta64(1, 'W') - x.date_last) / np.timedelta64(1, 'D')).astype(np.int),
                days_since_first = lambda x : ((x.week + np.timedelta64(1, 'W') - x.date_last.iloc[0])/ np.timedelta64(1, 'D')).astype(np.int),
                sum_in = lambda x : x.groupby('customer_id').sum_in.fillna(0),
                sum_out = lambda x : x.groupby('customer_id').sum_out.fillna(0),
                cum_sum_in = lambda x: x.groupby('customer_id').cum_sum_in.fillna(method='ffill'),
                cum_sum_out = lambda x: x.groupby('customer_id').cum_sum_out.fillna(method='ffill'),
                count_in = lambda x : x.groupby('customer_id').count_in.fillna(0),
                count_out = lambda x : x.groupby('customer_id').count_out.fillna(0),
                cum_count_in = lambda x: x.groupby('customer_id').cum_count_in.fillna(method='ffill'),
                cum_count_out = lambda x: x.groupby('customer_id').cum_count_out.fillna(method='ffill'),
                cum_count = lambda x : (x.cum_count_in + x.cum_count_out),
                balance = lambda x : x.cum_sum_in - x.cum_sum_out,
                count_tnx_type = lambda x : x.groupby('customer_id').count_tnx_type.fillna(0),
                week_churn = lambda x: np.where(x.count_in + x.count_out == 0, 1, 0),
                sum_in_relative = lambda x : x.groupby('customer_id').sum_in_relative.fillna(0),
                sum_out_relative = lambda x : x.groupby('customer_id').sum_out_relative.fillna(0),
                count_in_relative = lambda x : x.groupby('customer_id').count_in_relative.fillna(0),
                count_out_relative = lambda x : x.groupby('customer_id').count_out_relative.fillna(0),
                week_of_first_transaction = lambda x : x.groupby('customer_id').week_of_first_transaction.fillna(0),
                week_of_last_transaction = lambda x : x.groupby('customer_id').week_of_last_transaction.fillna(0),
                n_transaction_weekly = lambda x : (x.count_in + x.count_out),
                active_week = lambda x : np.where(x.n_transaction_weekly > 0, 1, 0),
                count_incoming_bigger_than_100 = lambda x : x.groupby('customer_id').count_incoming_bigger_than_100.fillna(0),
                count_incoming_bigger_than_500 = lambda x : x.groupby('customer_id').count_incoming_bigger_than_500.fillna(0),
                count_incoming_bigger_than_1000 = lambda x : x.groupby('customer_id').count_incoming_bigger_than_1000.fillna(0),
                cum_n_incoming_bigger_than_100 = lambda x : x.groupby('customer_id').cum_n_incoming_bigger_than_100.fillna(0),
                cum_n_incoming_bigger_than_500 = lambda x : x.groupby('customer_id').cum_n_incoming_bigger_than_500.fillna(0),
                cum_n_incoming_bigger_than_1000 = lambda x : x.groupby('customer_id').cum_n_incoming_bigger_than_1000.fillna(0),
                cum_monday = lambda x: x.groupby('customer_id').cum_monday.fillna(method='ffill'),
                monday = lambda x : x.groupby('customer_id').monday.fillna(0),
                cum_tuesday = lambda x: x.groupby('customer_id').cum_tuesday.fillna(method='ffill'),
                tuesday = lambda x : x.groupby('customer_id').tuesday.fillna(0),
                cum_wednesday = lambda x: x.groupby('customer_id').cum_wednesday.fillna(method='ffill'),
                wednesday = lambda x : x.groupby('customer_id').wednesday.fillna(0),
                cum_thursday = lambda x: x.groupby('customer_id').cum_thursday.fillna(method='ffill'),
                thursday = lambda x : x.groupby('customer_id').thursday.fillna(0),
                cum_friday = lambda x: x.groupby('customer_id').cum_friday.fillna(method='ffill'),
                friday = lambda x : x.groupby('customer_id').friday.fillna(0),
                cum_saturday = lambda x: x.groupby('customer_id').cum_saturday.fillna(method='ffill'),
                saturday = lambda x : x.groupby('customer_id').saturday.fillna(0),
                cum_sunday = lambda x: x.groupby('customer_id').cum_sunday.fillna(method='ffill'),
                sunday = lambda x : x.groupby('customer_id').sunday.fillna(0)
                )
                .drop(columns=['ij', 'week_min']))

            #organise df
            new_order = ['customer_id','week', 'month','n_week',
                 'days_to_next_max', 'days_since_last_at_end_of_the_week', 'days_since_first', #days
                 'sum_in', 'sum_out','cum_sum_in', 'cum_sum_out', 'count_in', 'count_out', 'cum_count_in','cum_count_out','cum_count','n_transaction_weekly','balance', #transaction and money
                 'count_incoming_bigger_than_100', 'count_incoming_bigger_than_500', 'count_incoming_bigger_than_1000', #big transaction monthly
                 'cum_n_incoming_bigger_than_100', 'cum_n_incoming_bigger_than_500', 'cum_n_incoming_bigger_than_1000', #big transaction cumulative
                 'cum_monday', 'monday', 'cum_tuesday', 'tuesday', 'cum_wednesday', 'wednesday', 'cum_thursday',#days of the week
                 'thursday', 'cum_friday', 'friday', 'cum_saturday', 'saturday', 'cum_sunday', 'sunday',
                 'sum_in_relative', 'sum_out_relative', 'count_in_relative','count_out_relative', #relative transaction
                 'count_tnx_type', #transaction type
                 'week_of_first_transaction', 'week_of_last_transaction', #month
                 'week_churn','active_week', 'days_in_month']#active or churner

            self.df_week = self.dfs[new_order].round(2)


#--------------------------------------------------------------------
# Historycal Df with month stats *** Historycal Df with month stats
#--------------------------------------------------------------------
    def make_history_df(self):

        if self.agg == 'month':

            self.history_month = (
                self.df_month
                .sort_values(['month', 'customer_id'])
                .assign(
                    n_active_months = lambda x : x.groupby('customer_id').active_months.cumsum(),
                    n_churn_months = lambda x : x.groupby('customer_id').churn.cumsum(),
                    n_tnx_type = lambda x : x.groupby('customer_id').count_tnx_type.cumsum(),
                    cumulative_balance = lambda x : x.groupby('customer_id').balance.cumsum(),
                    )
                .groupby('customer_id', as_index = False)
                .agg({
                    'n_months':[last_value],
                    'n_active_months':[last_value],
                    'n_churn_months':[last_value],
                     'cum_sum_in':[last_value],
                     'cum_sum_out': [last_value],
                     'cum_count_in':[last_value],
                     'cum_count_out':[last_value],
                     'cum_count':[last_value],
                     'active_months' : [last_value],
                     'n_tnx_type':[last_value],
                     'cumulative_balance':[last_value],
                     'balance':[last_value],
                     'cum_n_incoming_bigger_than_100' : [last_value],
                     'cum_n_incoming_bigger_than_500': [last_value],
                     'cum_n_incoming_bigger_than_1000': [last_value],
                    'churn':[last_value]
                                }))

            self.history_month.columns = ['customer_id','tot_months', 'n_active_months', 'n_churn_months',
                                'total_amount_income', 'total_amount_outgoing',
                                'total_n_transaction_incoming', 'total_n_transaction_outgoing',
                                'total_n_transaction',
                                'currently_active',
                                'cumulative_tnx_grouped',
                                'cumulative_balance', 'currently_balance',
                                 'cumulative_transaction_bigger_than_100_in','cumulative_transaction_bigger_than_500_in', 'cumulative_transaction_bigger_than_1000_in',
                                         'target']

            self.history_month = (
                    self.history_month
                    .assign(
                        avg_sum_in_per_active_months = lambda x : x.total_amount_income / x.n_active_months,
                        avg_sum_out_per_active_months = lambda x : x.total_amount_outgoing / x.n_active_months,
                        avg_count_in_per_active_months = lambda x : x.total_n_transaction_incoming / x.n_active_months,
                        avg_count_out_per_active_months = lambda x : x.total_n_transaction_outgoing / x.n_active_months,
                        avg_count_per_active_months = lambda x : x.total_n_transaction / x.n_active_months,
                        avg_tnx_detailed_months = lambda x : x.cumulative_tnx_grouped / x.n_active_months,
                        avg_balance_at_end_of_month = lambda x : x.cumulative_balance / x.n_active_months,
                        avg_transactions_bigger_than_100 = lambda x : x.cumulative_transaction_bigger_than_100_in / x.n_active_months,
                        avg_transactions_bigger_than_500 = lambda x : x.cumulative_transaction_bigger_than_500_in / x.n_active_months,
                        avg_transactions_bigger_than_1000 = lambda x : x.cumulative_transaction_bigger_than_1000_in / x.n_active_months,
                        )).drop(['cumulative_tnx_grouped', 'cumulative_balance'], axis=1).round(2)

#--------------------------------------------------------------------
# Historycal Df with week stats *** Historycal Df with week stats
#--------------------------------------------------------------------
        if self.agg == 'week':

            self.history_week = (
                self.df_week
                .sort_values(['week', 'customer_id'])
                .assign(
                    n_active_weeks = lambda x : x.groupby('customer_id').active_week.cumsum(),
                    n_churn_weeks = lambda x : x.groupby('customer_id').week_churn.cumsum(),
                    n_tnx_type = lambda x : x.groupby('customer_id').count_tnx_type.cumsum(),
                    cumulative_balance = lambda x : x.groupby('customer_id').balance.cumsum(),

                    )
                .groupby('customer_id', as_index = False)
                .agg({
                    'n_week':[last_value],
                    'n_active_weeks':[last_value],
                    'n_churn_weeks':[last_value],
                    'cum_sum_in':[last_value],
                    'cum_sum_out': [last_value],
                    'cum_count_in':[last_value],
                    'cum_count_out':[last_value],
                    'cum_count':[last_value],
                    'active_week' : [last_value],
                    'n_tnx_type':[last_value],
                    'cumulative_balance':[last_value],
                    'balance':[last_value],
                    'cum_n_incoming_bigger_than_100' : [last_value],
                    'cum_n_incoming_bigger_than_500': [last_value],
                    'cum_n_incoming_bigger_than_1000': [last_value],
                                }))


            self.history_week.columns = ['customer_id','tot_weeks', 'n_active_weeks', 'n_churn_weeks',
                               'total_amount_income', 'total_amount_outgoing',
                               'total_n_transaction_incoming', 'total_n_transaction_outgoing',
                               'total_n_transaction',
                               'currently_active',
                               'cumulative_tnx_grouped',
                               'cumulative_balance', 'currently_balance',
                                'cumulative_transaction_bigger_than_100_in','cumulative_transaction_bigger_than_500_in', 'cumulative_transaction_bigger_than_1000_in']

            self.history_week = (
                    self.history_week
                    .assign(
                        avg_sum_in_per_active_weeks = lambda x : x.total_amount_income / x.n_active_weeks,
                        avg_sum_out_per_active_weeks = lambda x : x.total_amount_outgoing / x.n_active_weeks,
                        avg_count_in_per_active_weeks = lambda x : x.total_n_transaction_incoming / x.n_active_weeks,
                        avg_count_out_per_active_weeks = lambda x : x.total_n_transaction_outgoing / x.n_active_weeks,
                        avg_count_per_active_weeks = lambda x : x.total_n_transaction / x.n_active_weeks,
                        avg_tnx_detailed_weeks = lambda x : x.cumulative_tnx_grouped / x.n_active_weeks,
                        avg_balance_at_end_of_week = lambda x : x.cumulative_balance / x.n_active_weeks,
                        avg_transactions_bigger_than_100 = lambda x : x.cumulative_transaction_bigger_than_100_in / x.n_active_weeks,
                        avg_transactions_bigger_than_500 = lambda x : x.cumulative_transaction_bigger_than_500_in / x.n_active_weeks,
                        avg_transactions_bigger_than_1000 = lambda x : x.cumulative_transaction_bigger_than_1000_in / x.n_active_weeks,
                        )).drop(['cumulative_tnx_grouped', 'cumulative_balance'], axis=1).round(2)


    def write_file(self, df = 'aggregate', how = 'month', type_format = 'csv', folder_name = 'output'):

        """
        df = ['history', 'aggregate']
        how = ['month', 'week']
        type_format = ['csv', 'parquet']
        """

        if not os.path.exists(folder_name):
            os.makedirs(folder_name)

        if df == 'aggregate':
            if how == 'month':
                if type_format == 'csv':
                    self.df_month.to_csv(folder_name + '/aggregate_month.csv', index=False)
                elif type_format == 'parquet':
                    self.df_month.to_parquet(folder_name + '/aggregate_month.parquet.gzip', index=False)
                else:
                    print('Unknown format')

            elif how == 'week':
                if type_format == 'csv':
                    self.df_week.to_csv(folder_name + '/aggregate_month.csv', index=False)
                elif type_format == 'parquet':
                    self.df_week.to_parquet(folder_name + '/aggregate_week.parquet.gzip', index=False)
                else:
                    print('Unknown format')

            else:
                print('Unknown method of aggregation')

        elif df == 'history':
            if how == 'month':
                if type_format == 'csv':
                    self.history_month.to_csv(folder_name + '/history_by_month.csv', index=False)
                elif type_format == 'parquet':
                    self.history_month.to_parquet(folder_name + '/history_by_month.parquet.gzip', index=False)
                else:
                    print('Unknown format')

            elif how == 'week':
                if type_format == 'csv':
                    self.history_week.to_csv(folder_name + '/history_by_week.csv', index=False)
                elif type_format == 'parquet':
                    self.history_week.to_parquet(folder_name + '/history_by_week.parquet.gzip', index=False)
                else:
                    print('Unknown format')

            else:
                print('Unknown method of aggregation')


# Use case

In [8]:
C = Aggregation_Transaction('file_with_random_transaction.csv', 'csv', 'month')


In [9]:
C.df_month.head()

,customer_id,month,n_months,days_to_next_max,days_since_last_at_end_of_the_month,days_since_first,sum_in,sum_out,cum_sum_in,cum_sum_out,count_in,count_out,cum_count_in,cum_count_out,cum_count,n_transaction_monthly,balance,count_incoming_bigger_than_100,count_incoming_bigger_than_500,count_incoming_bigger_than_1000,cum_n_incoming_bigger_than_100,cum_n_incoming_bigger_than_500,cum_n_incoming_bigger_than_1000,cum_monday,monday,cum_tuesday,tuesday,cum_wednesday,wednesday,cum_thursday,thursday,cum_friday,friday,cum_saturday,saturday,cum_sunday,sunday,sum_in_relative,sum_out_relative,count_in_relative,count_out_relative,count_tnx_type,month_of_first_transaction,month_of_last_transaction,churn,active_months,days_in_month
0,1,2018-01-01,1,6.0,2,2,16.06,0.00,16.06,0.00,1.0,0.0,1.0,0.0,1.0,1.0,16.06,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.00,1.00,0.00,1.0,1.0,0.0,0,1,31
6672,503,2018-01-01,1,1.0,0,2,15.49,116.15,15.49,116.15,1.0,6.0,1.0,6.0,7.0,7.0,-100.66,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,3.0,3.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.12,0.88,0.14,0.86,3.0,1.0,0.0,0,1,31
600,51,2018-01-01,1,2.0,0,2,0.00,122.41,0.00,122.41,0.0,7.0,0.0,7.0,7.0,7.0,-122.41,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.00,1.00,0.00,1.00,3.0,1.0,0.0,0,1,31
6648,502,2018-01-01,1,1.0,0,2,0.00,231.54,0.00,231.54,0.0,11.0,0.0,11.0,11.0,11.0,-231.54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,0.00,1.00,2.0,1.0,0.0,0,1,31
6624,501,2018-01-01,1,1.0,0,2,23.20,0.00,23.20,0.00,1.0,0.0,1.0,0.0,1.0,1.0,23.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.00,1.00,0.00,1.0,1.0,0.0,0,1,31


In [10]:
C.make_history_df()

In [11]:
C.history_month.head()

,customer_id,tot_months,n_active_months,n_churn_months,total_amount_income,total_amount_outgoing,total_n_transaction_incoming,total_n_transaction_outgoing,total_n_transaction,currently_active,currently_balance,cumulative_transaction_bigger_than_100_in,cumulative_transaction_bigger_than_500_in,cumulative_transaction_bigger_than_1000_in,target,avg_sum_in_per_active_months,avg_sum_out_per_active_months,avg_count_in_per_active_months,avg_count_out_per_active_months,avg_count_per_active_months,avg_tnx_detailed_months,avg_balance_at_end_of_month,avg_transactions_bigger_than_100,avg_transactions_bigger_than_500,avg_transactions_bigger_than_1000
0,1,24,24,0,6942.97,5884.31,147.0,128.0,275.0,1,1058.66,0.0,0.0,0.0,0,289.29,245.18,6.12,5.33,11.46,2.54,575.68,0.0,0.0,0.0
1,2,17,8,9,238.76,360.13,6.0,11.0,17.0,1,-121.37,0.0,0.0,0.0,0,29.84,45.02,0.75,1.38,2.12,1.62,-67.90,0.0,0.0,0.0
2,3,18,1,17,0.00,66.37,0.0,1.0,1.0,0,-66.37,0.0,0.0,0.0,1,0.00,66.37,0.00,1.00,1.00,1.00,-1194.66,0.0,0.0,0.0
3,4,23,6,17,0.00,204.05,0.0,6.0,6.0,0,-204.05,0.0,0.0,0.0,1,0.00,34.01,0.00,1.00,1.00,1.00,-591.35,0.0,0.0,0.0
4,5,12,1,11,90.66,108.72,1.0,2.0,3.0,0,-18.06,0.0,0.0,0.0,1,90.66,108.72,1.00,2.00,3.00,2.00,-216.72,0.0,0.0,0.0


In [12]:
C.write_file('aggregate', 'month', 'csv')